Pattern -> Descriptors -> Embeddings -> Coordinates (by patt index)

  -->  MIDI Drum Assignment -> Notes In Channels -> Flattening Type Results (by patt index) ==> for all flattening types.
  
---
Build and test NN for each (flat, coord) combo & report MAE results.

In [3]:
## Imports ##
import os
import numpy as np
import sys
import random

## TODO: double check these need to be imported in this file
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as tdata
import torch.backends.mps
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
## File Locations (GitHub repo dir) ##
dir=os.getcwd()
os.listdir(dir)

import descriptors as desc
import functions as func
import neuralnetwork as nn

In [ ]:
## File Locations (Google Drive) ##
#drive.mount('/content/drive/MyDrive/Tapping_Force_Experiment_Data')

'''
Mounting my drive doesn't work by code, but does by side panel.
We will need to figure this
'''
dir = os.chdir('/content/drive/MyDrive/Tapping_Force_Experiment_Data')
os.listdir(dir)
import descriptors as desc
import functions as func

In [ ]:
## Patterns -> Descriptors & Embedding Coordinates ##


In [ ]:
## Convert Patterns (Full MIDI --> (8?) 3-Channel) ##

In [ ]:
## Convert 3-channel to all 13 flat types. ##

In [ ]:
## Per selected Flat: NN build + (train + test)-10fold ##
# 3 data representation types: Continuous, Continuous Above Mean, Discrete #




# TODO:
# Add 10 fold cross evaluation in NN build pipeline
# https://github.com/christianversloot/machine-learning-articles/blob/main/how-to-use-k-fold-cross-validation-with-keras.md
# Return results and move print to this file



In [ ]:
## For all Flat: NN build + (train + test)-10fold ## WARNING: LONGTIME!